In [1]:
#@title Install Packages

In [2]:
!pip install -q transformers
!pip install -q torchinfo
!pip install -q datasets
!pip install -q evaluate
!pip install -q nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 3.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-

In [3]:
!sudo apt-get update
! sudo apt-get install tree

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,381 kB]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [70.9 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,808 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu ja

In [4]:
#@title Imports

import numpy as np

import transformers
import evaluate

import nltk

from datasets import load_dataset
from torchinfo import summary

from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer

In [5]:
# @title Mount Google Drive

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
dir_root = '/content/drive/MyDrive/266-final/'
# dir_data = '/content/drive/MyDrive/266-final/data/'
dir_data = '/content/drive/MyDrive/266-final/data/zenodo/'
dir_models = '/content/drive/MyDrive/266-final/models/'
dir_results = '/content/drive/MyDrive/266-final/results/'

In [8]:
!tree -L 2 /content/drive/MyDrive/266-final/

/content/drive/MyDrive/266-final/
├── data
│   ├── train-dev-no-test
│   └── zenodo
├── models
├── notebook-scripts
│   ├── 1.0 Propaganda Dataset Classification Functional Test MVP with BERT base.ipynb
│   ├── 1.1 Propaganda Dataset Classification Functional Test MVP with BERT base.ipynb
│   └── Experiments
├── paper
├── results
└── slides

9 directories, 2 files


In [26]:
# !tree -L 4 /content/drive/MyDrive/266-final/

In [9]:
#@title Import Data

In [10]:
!ls -li /content/drive/MyDrive/266-final/data/zenodo/

total 375
50 drwx------ 2 root root   4096 Apr  3 23:24 dev-articles
46 -rw------- 1 root root  22850 Dec 11  2019 dev-task-TC-template.out
47 -rw------- 1 root root   4886 Dec 11  2019 README.md
51 drwx------ 2 root root   4096 Apr  3 23:24 train-articles
48 drwx------ 2 root root   4096 Apr  3 23:24 train-labels-task1-span-identification
49 drwx------ 2 root root   4096 Apr  3 23:24 train-labels-task2-technique-classification
44 -rw------- 1 root root 108269 Dec 11  2019 train-task1-SI.labels
45 -rw------- 1 root root 230658 Dec 11  2019 train-task2-TC.labels


In [11]:
# dataset = load_dataset("sem_eval_2020_task_11") # does not work, revert to manual load

In [12]:
import os
import pandas as pd

DATA_DIR = "/content/drive/MyDrive/266-final/data/zenodo/"

# Directories
TRAIN_ARTICLES_DIR = os.path.join(DATA_DIR, "train-articles")
DEV_ARTICLES_DIR   = os.path.join(DATA_DIR, "dev-articles")

# Label directories (if you need them at the directory level)
TRAIN_LABELS_TASK1_DIR = os.path.join(DATA_DIR, "train-labels-task1-span-identification")
TRAIN_LABELS_TASK2_DIR = os.path.join(DATA_DIR, "train-labels-task2-technique-classification")

# Individual label files
TRAIN_TASK1_LABELS_FILE = os.path.join(DATA_DIR, "train-task1-SI.labels")
TRAIN_TASK2_LABELS_FILE = os.path.join(DATA_DIR, "train-task2-TC.labels")

# Dev task template (if needed)
DEV_TC_TEMPLATE_FILE = os.path.join(DATA_DIR, "dev-task-TC-template.out")

In [13]:
def load_articles_from_directory(directory):
    """
    Reads all files from the given directory and returns
    a list of dicts: [{"filename": ..., "text": ...}, ...]
    """
    articles = []
    for filename in sorted(os.listdir(directory)):
        filepath = os.path.join(directory, filename)
        if os.path.isfile(filepath) and filename.endswith(".txt"):
            with open(filepath, "r", encoding="utf-8") as f:
                text = f.read()
            articles.append({
                "filename": filename,
                "text": text
            })
    return articles

train_articles_list = load_articles_from_directory(TRAIN_ARTICLES_DIR)
dev_articles_list   = load_articles_from_directory(DEV_ARTICLES_DIR)

train_articles_df = pd.DataFrame(train_articles_list)
dev_articles_df   = pd.DataFrame(dev_articles_list)

print("Number of training articles:", len(train_articles_df))
print("Number of dev articles:", len(dev_articles_df))

Number of training articles: 371
Number of dev articles: 75


In [14]:
train = train_articles_df
dev   = dev_articles_df

In [15]:
train_task1_labels = pd.read_csv(TRAIN_TASK1_LABELS_FILE, sep="\t", header=None)
train_task2_labels = pd.read_csv(TRAIN_TASK2_LABELS_FILE, sep="\t", header=None)

print(train_task1_labels.head())
print(train_task2_labels.head())

           0     1     2
0  111111111   265   323
1  111111111  1795  1935
2  111111111   149   157
3  111111111  1069  1091
4  111111111  1334  1462
           0                         1     2     3
0  111111111       Appeal_to_Authority   265   323
1  111111111       Appeal_to_Authority  1795  1935
2  111111111                     Doubt   149   157
3  111111111                Repetition  1069  1091
4  111111111  Appeal_to_fear-prejudice  1334  1462


In [16]:
train = {
    "articles": train_articles_df,
    "task1_labels": train_task1_labels,
    "task2_labels": train_task2_labels
}

dev_task_tc_template = pd.read_csv(DEV_TC_TEMPLATE_FILE, sep="\t", header=None)
dev = {
    "articles": dev_articles_df,
    "tc_template": dev_task_tc_template
}

In [17]:
print("Training data keys:", train.keys())
print("First few train articles:\n", train["articles"].head(), "\n")

print("First few train Task1 labels:\n", train["task1_labels"].head(), "\n")
print("First few train Task2 labels:\n", train["task2_labels"].head(), "\n")

print("Dev data keys:", dev.keys())
print("First few dev articles:\n", dev["articles"].head(), "\n")
print("Dev TC template:\n", dev["tc_template"].head(), "\n")

Training data keys: dict_keys(['articles', 'task1_labels', 'task2_labels'])
First few train articles:
                filename                                               text
0  article111111111.txt  Next plague outbreak in Madagascar could be 's...
1  article111111112.txt  US bloggers banned from entering UK\n\nTwo pro...
2  article111111113.txt  Kate Steinle's death at the hands of a Mexican...
3  article111111114.txt  U.S. judge frees Indonesian immigrant held by ...
4  article111111115.txt  Here are all the sexual misconduct accusations... 

First few train Task1 labels:
            0     1     2
0  111111111   265   323
1  111111111  1795  1935
2  111111111   149   157
3  111111111  1069  1091
4  111111111  1334  1462 

First few train Task2 labels:
            0                         1     2     3
0  111111111       Appeal_to_Authority   265   323
1  111111111       Appeal_to_Authority  1795  1935
2  111111111                     Doubt   149   157
3  111111111               

In [18]:
# train_articles_df.head()

In [19]:
train_articles_df.shape

(371, 2)

In [20]:
dev_articles_df.shape

(75, 2)

In [21]:
train_task2_labels.shape

(6129, 4)

In [22]:
train_task2_labels.head()

,0,1,2,3
0,111111111,Appeal_to_Authority,265,323
1,111111111,Appeal_to_Authority,1795,1935
2,111111111,Doubt,149,157
3,111111111,Repetition,1069,1091
4,111111111,Appeal_to_fear-prejudice,1334,1462


In [23]:
dev_

NameError: name 'dev_' is not defined